<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/river_width.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install geemap

In [ ]:
import ee
import geemap
from geemap.algorithms import river

Create an interactive map.

In [ ]:
Map = geemap.Map()
Map

Find an image by ROI.

In [ ]:
point = ee.Geometry.Point([-88.08, 37.47])

In [ ]:
image = (
    ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
    .filterBounds(point)
    .sort("CLOUD_COVER")
    .first()
)

Add image to the map.

In [ ]:
Map.addLayer(image, {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}, "Landsat")
Map.centerObject(image)

Extract river width for a single image.

In [ ]:
river.rwc(image, folder="export", water_method='Jones2019')

Add result to the map.

In [ ]:
fc = ee.FeatureCollection("users/giswqs/public/river_width")
Map.addLayer(fc, {}, "River width")

Add [Global River Width Dataset](https://samapriya.github.io/awesome-gee-community-datasets/projects/grwl) to the map. 

In [ ]:
water_mask = ee.ImageCollection(
    "projects/sat-io/open-datasets/GRWL/water_mask_v01_01"
).median()
Map.addLayer(water_mask, {'min': 11, 'max': 125, 'palette': 'blue'}, 'GRWL Water Mask')

In [ ]:
grwl_water_vector = ee.FeatureCollection(
    "projects/sat-io/open-datasets/GRWL/water_vector_v01_01"
)
Map.addLayer(
    grwl_water_vector.style(**{'fillColor': '00000000', 'color': 'FF5500'}),
    {},
    'GRWL Vector',
)

Find images by ROI.

In [ ]:
images = (
    ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
    .filterBounds(point)
    .sort("CLOUD_COVER")
    .limit(3)
)

Get the list of image ids.

In [ ]:
ids = images.aggregate_array("LANDSAT_ID").getInfo()
ids

Extract river width for a list of images.

In [ ]:
river.rwc_batch(ids, folder="export", water_method='Jones2019')